#RESTORE COMMAND

In [0]:
from delta.tables import *

In [0]:
DeltaTable.createOrReplace(spark) \
    .tableName("restore_table") \
    .addColumn("id", "INT") \
    .addColumn("name", "STRING") \
    .addColumn("gender", "STRING") \
    .addColumn("salary", "INT") \
    .addColumn("dept", "STRING") \
    .location('dbfs:/FileStore/tables/delta/restore_table') \
    .execute()

Out[10]: <delta.tables.DeltaTable at 0x7fd074f7dd90>

In [0]:
%sql

INSERT INTO restore_table VALUES (1, "Carl Mike", "m", 170000, "Support");

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

INSERT INTO restore_table (id, name, gender, salary, dept) VALUES
(2, "Mikel Clark", "m", 254300, "IT"),
(3, "Bob Smith", "m", 220000, "IT");

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql

INSERT INTO restore_table (id, name, gender, salary, dept) VALUES
(4, "Mary Scala", "f", 230000, "Sales");

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

INSERT INTO restore_table (id, name, gender, salary, dept) VALUES
(5, "Susan Liam", "f", 150000, "Sales");

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

DELETE FROM restore_table WHERE id=5 OR id=1;

num_affected_rows
2


In [0]:
%sql

SELECT * FROM restore_table

id,name,gender,salary,dept
2,Mikel Clark,m,254300,IT
3,Bob Smith,m,220000,IT
4,Mary Scala,f,230000,Sales


In [0]:
instance_dt = DeltaTable.forPath(spark,'dbfs:/FileStore/tables/delta/restore_table')

In [0]:
instance_dt.history().show()

+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|           operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      5|2024-08-26 22:00:46|8904480319608266|andresmunozpampil...|              DELETE|{predicate -> ["(...|null|{2925731615638739}|0826-214512-psjg5zrb|          4|WriteSerializable|        false|{numRemovedFiles ...|        null|Databr

## Restore Using Version

In [0]:
instance_dt.restoreToVersion(4)

Out[23]: DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [0]:
%sql

SELECT * FROM restore_table

id,name,gender,salary,dept
2,Mikel Clark,m,254300,IT
3,Bob Smith,m,220000,IT
1,Carl Mike,m,170000,Support
5,Susan Liam,f,150000,Sales
4,Mary Scala,f,230000,Sales


## Restore Using Timestamp

In [0]:
instance_dt.restoreToTimestamp("2024-08-26 22:00:14")

Out[25]: DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [0]:
%sql

SELECT * FROM restore_table

id,name,gender,salary,dept
1,Carl Mike,m,170000,Support


In [0]:
instance_dt.history().show()

+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|           operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      7|2024-08-26 22:07:33|8904480319608266|andresmunozpampil...|             RESTORE|{version -> null,...|null|{2925731615638739}|0826-214512-psjg5zrb|          6|     Serializable|        false|{numRestoredFiles...|        null|Databr